<h2>Using the API to Query Planned JWST Observations in MAST</h2>
<p><i>By Peter Forshay (pforshay@stsci.edu)</i></p>
<p>In this Notebook we'll take a look at how to use the MAST API to find planned JWST observations from GTO and ERS programs.  We'll start with a basic filtered query to find all planned observations before moving on to constructing more specific requests:</p>
<ul>
    <li><a href=#search-by-pos>Searching by position</a></li>
    <li><a href=#analyzing-results>Analyzing our results</a></li>
    <li><a href=#save-to-file>Saving results to a file</a></li>
    <li><a href=#query-multi-targs>Querying multiple positions at once</a></li>
    <li><a href=#search-by-name>Resolving stationary target names</a></li>
    <li><a href=#moving-targets>Searching for moving target names</a></li>
    <li><a href=#send-to-aladin>Visualizing results using Aladin</a></li>
</ul>
<p>Be aware that MAST queries can take some time to run and get a response, so please be patient when running cells back to back.  If you encounter any errors, try to run the cell again when the previous cells have completed execution.</p>
<p><b>IMPORTANT DISCLAIMER:</b>  To avoid unintentional duplications, JWST proposers are required to check their proposed observations against those already approved.  The tools provided here may be able to assist a user in identifying POTENTIAL observing conflicts for JWST programs.  It remains the PI's responsibility to determine whether or not the result provided by these tools constitutes an actual conflict, in accordance with the <a href="https://jwst-docs.stsci.edu/display/JSP/JWST+Duplicate+Observations+Policy" target="_blank">JWST Duplicate Observations Policy</a>.</p>
<p><i>(Much of this code is based on the tutorials and examples provided in the <a href="https://mast.stsci.edu/api/v0/" target="_blank">MAST API documentation pages found here</a>)</i></p>

<h3>Standard import statements for a MAST Mashup API request</h3>

In [1]:
import sys
import os
import time
import re
import json

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib   

from astropy.table import Table
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)

<h3>Define the MAST query module to handle appropriate formatting</h3>

In [2]:
def mastQuery(request):
    """Perform a MAST query.
    
    Parameters
    ----------
    request : dict
        The MAST request json object
    
    Returns
    -------
    head, content 
        Head is the response HTTP headers, and content is the returned data
    """
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

<h3>Here is where we begin to customize the code to perform our own queries:</h3>
<p>The first search below sends a basic filtered query to count all planned observations, designated by a calibration level of -1.  In the 'data' entry, we can see 'Column1' returns with the number of results for the submitted query.</p>
<ul>
    <li>"paramName":"calib_level" allows us to filter based on calibration level.</li>
    <li>"values":["-1"] defines a planned observation.</li>
</ul>
<p>Additional parameters to filter on:</p>
<ul>
    <li>target_name</li>
    <li>s_ra</li>
    <li>s_dec</li>
    <li>instrument</li>
    <li>filters</li>
    <li>proposal_id</li>
    <li>target_classification</li>
</ul>

In [83]:
mashupRequest = {"service":"Mast.Caom.Filtered",
                 "format":"json",
                 "params":{"columns":"COUNT_BIG(*)",    # "COUNT_BIG(*)" will only get a count of the results
                           "filters":[{"paramName":"calib_level",
                                       "values":["-1"]
                                      },
                                      {"paramName":"obs_collection",
                                       "values":["JWST"]
                                      }]
                          }
                }
    
headers,outString = mastQuery(mashupRequest)
queryResults = json.loads(outString)

pp.pprint(queryResults)

{   'data': [{'Column1': 13727}],
    'fields': [{'name': 'Column1', 'type': 'string'}],
    'msg': '',
    'paging': {   'page': 1,
                  'pageSize': 1,
                  'pagesFiltered': 1,
                  'rows': 1,
                  'rowsFiltered': 1,
                  'rowsTotal': 1},
    'status': 'COMPLETE'}


<p>A succesful query should return a dictionary with a 'status': 'COMPLETE' entry.  The 'data' from this query should simply contain a count of all the planned observations for JWST present in MAST's CAOM (Common Archive Observation Model) database.</p>

<p>Next, we will establish a script that will construct a Mashup request given a number of different filters.</p>

In [130]:
def submit_mashup_request(filters, coordinates=None, radius=0.2, count=True):
    """
    Construct a filtered mashup request to send to the mastQuery function.  Return 
    either the results of the query or just the results count.
    
    Parameters
    ----------
    filters : dict
        A dictionary of query filters to use in a mashup request.
    coordinates : tuple, optional
        Expects a pair of coordinates in degrees, if querying a position.
        (default=None)
    radius : float, optional
        Defines the radius to search around the designated coordinates within.
        Also in degrees.  (default=0.2)
    count : bool, optional
        Flag to designate whether a full query is submitted, or just the count
        results.  (default=True)
        
    Returns
    -------
    queryResults : dict
        A dictionary of query results with keys of 'data', 'fields', 'msg',
        'paging', and 'status'.
    """
    
    # Construct the list of filters to query on.
    filt_list = []
    for f, val in filters.items():
        vals = val if isinstance(val, list) else [val]
        entry = {"paramName": f,
                 "values": vals
                }
        if f == "filters":
            entry["seperators"] = ";"
        filt_list.append(entry)
    
    # Update settings if coordinates are provided.
    if isinstance(coordinates, tuple):
        service = "Mast.Caom.Filtered.Position"
        ra_deg, dec_deg = coordinates
        if not isinstance(radius, float):
            err = "A <float> search radius is needed for position queries"
            raise TypeError(err)
    elif coordinates is None:
        service = "Mast.Caom.Filtered"
    else:
        print("'submit_mashup_request()' expects coordinates in a tuple")
        return
    
    # Update settings for retrieving full query results or results count.
    if count:
        columns = "COUNT_BIG(*)"    # "COUNT_BIG(*)" will only get a count of the results
    else:
        columns = "*"
        
    # Construct the mashup request.
    params_dict = {"columns": columns, "filters": filt_list}
    if coordinates:
        position = "{0}, {1}, {2}".format(ra_deg, dec_deg, radius)
        params_dict["position"] = position
    mashupRequest = {"service":service,
                     "format":"json",
                     "params":params_dict
                     }

    # Execute the query.
    headers, outString = mastQuery(mashupRequest)
    queryResults = json.loads(outString)
    
    # Analyze the results count if counting.
    if count:
        data = queryResults['data']
        result_count = data[0]['Column1']
        if result_count == 0:
            raise RuntimeError("No matching results found")
        elif result_count > 50000:
            raise RuntimeError("Query returned {} results (>50k limit)".format(result_count))
        else:
            # Rerun the full query if result counts are acceptable.
            return submit_mashup_request(filters, 
                                         coordinates=coordinates,
                                         radius=radius,
                                         count=False
                                        )
    # Otherwise return the query results.
    else:
        print("Returned {} query results".format(len(queryResults['data'])))
        return queryResults

<p>This function allows us to easily retrieve all planned JWST observation information (as long as there are fewer than 50k) with the following:</p> 

In [131]:
MY_FILTS = {"calib_level": "-1", "obs_collection": "JWST"}
ALL_RESULTS = submit_mashup_request(MY_FILTS)

Returned 13727 query results


<a id="search-by-pos"></a>
<h3>Searching by position</h3>
<p>Next, we can begin adding additional filters to narrow our query, starting with position.  First off, in order to send a filtered position search via the API, we'll need to format the position in question as an RA, DEC tuple in degrees.  Converting a set of coordinates to degrees is made pretty easy by using the <a href="http://docs.astropy.org/en/stable/coordinates/" target="_blank">astropy.coordinates SkyCoord class</a>.</p>

In [132]:
from astropy.coordinates import SkyCoord

def check_user_coordinates(*args):
    """
    Convert a single user coordinate pair to a tuple of degrees.  Able to accept multiple input formats, 
    relying on SkyCoord class.
    
    Parameters
    ----------
    *args : multiple
        A user may pass individual RA and DEC coordinates or as a single tuple.
        Coordinates may be floats or strings.
    
    Returns
    -------
    coords : tuple
        An RA, DEC tuple in degrees.
    """
    
    # Extract RA and Dec values from args.
    if len(args) == 1 and isinstance(args[0], tuple):
        ra, dec = args[0]
    elif len(args) == 2:
        ra = args[0]
        dec = args[1]
    else:
        err = "Could not parse coordinates: {}".format(args)
        raise ValueError(err)
        
    # Construct the tuple of formatted coordinates.
    if isinstance(ra, str) and isinstance(dec, str):
        ra = format_coordinate_str(ra, ['h', 'm', 's'])
        dec = format_coordinate_str(dec, ['d', 'm', 's'])
        obj = SkyCoord(ra, dec)
        ra_deg = obj.ra.deg
        dec_deg = obj.dec.deg
        coords = (ra_deg, dec_deg)
    elif isinstance(ra, float) and isinstance(dec, float):
        coords = (ra, dec)
    else:
        err = "Could not parse RA/DEC: {0}, {1}".format(ra, dec)
        raise TypeError(err)
    
    return coords

def format_coordinate_str(coord, units):
    """
    Make sure coordinate strings are in "**h**m**s"-like format.
    
    Parameters
    ----------
    coord : str
        An RA or Dec coordinate string.
    units : list
        A list of unit letters, either [h, m, s] or [d, m, s].
        
    Returns
    -------
    new : str
        A formatted coordinate string.
    """
    
    assert isinstance(coord, str)
    new = ""
    
    # Replace ':' with unit letters, if present.
    if ':' in coord:
        chunks = coord.split(':', 2)
        for n in range(len(chunks)):
            new = "".join([new, chunks[n], units[n]])
    else:
        new = coord
        
    # Check that each unit letter is present before returning string.
    for u in units:
        if u not in new:
            raise ValueError("Could not parse {}".format(coord))
    else:
        return new

In [133]:
# Select our coordinates
RA = '04:16:09.370'
DEC = '-24:04:20.50'
SAMPLE_COORDS = check_user_coordinates(RA, DEC)
print(SAMPLE_COORDS)

(64.03904166666666, -24.07236111111111)


<p>Now that we have our RA and Dec available in degrees, we can define a radius (also in degrees) and submit a filtered position query.  Recommended search radii for various JWST observing modes are provided within the <a href="https://jwst-docs.stsci.edu/display/JSP/JWST+Duplicate+Observations+Policy#JWSTDuplicateObservationsPolicy-DuplicationCheckingandReviewProcedures" target="_blank">JWST Duplication Policy information found on JDox</a>.  By default, we keep the "columns":"COUNT_BIG(*)" to simply return a count of the results, in case we hit a large number of entries.</p>

In [134]:
def filtered_position_query(filters, coordinates):
    """
    Construct a filtered position mashup request to send to the mastQuery module.  Return 
    either the results of the query or just the results count.
    
    Parameters
    ----------
    filters : dict
        Define the filters to apply to the query.
    coordinates : tuple
        Expects a pair of coordinates in degrees.
        
    Returns
    -------
    queryResults : dict
        A dictionary of query results with keys of 'data', 'fields', 'msg',
        'paging', and 'status'
    """
        
    # Make sure the filters dict is not empty.
    assert len(filters.keys()) > 0
    
    # Check the provided coordinates.
    coords = check_user_coordinates(*coordinates)
    
    queryResults = None
    try:
        queryResults = submit_mashup_request(filters, coordinates=coords)
    except RuntimeError as err:
        print("QUERY ERROR: {}".format(err))
        
    return queryResults

In [135]:
TEST_QUERY = filtered_position_query(MY_FILTS, SAMPLE_COORDS)

Returned 152 query results


<p>In the above result we see we get 152 matches for a search on JWST at this position.  We can now use the pandas DataFrame class for a better look at some of the information returned from our query.</p>

In [136]:
import pandas as pd

def convert_to_dataframe(queryResults):
    """Create a Pandas DataFrame from a MAST query results 'data' dictionary but just
    return the first few rows.
    
    Parameters
    ----------
    queryResults : dict
        The returned results from a MAST query.
        
    Returns
    -------
    frame : pandas.DataFrame
        The results as a DataFrame object.
    """
    
    frame = pd.DataFrame.from_dict(queryResults['data'])
    return frame

In [137]:
TEST_FRAME = convert_to_dataframe(TEST_QUERY)
TEST_FRAME[:4]

,calib_level,dataRights,dataURL,dataproduct_type,distance,em_max,em_min,filters,instrument_name,intentType,...,s_region,sequence_number,srcDen,t_exptime,t_max,t_min,t_obs_release,target_classification,target_name,wavelength_region
0,-1,None,None,spectrum,0.0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04159439266914 -24.09858928763...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
1,-1,None,None,image,0.0,2226000000000,1755000000000,F200W;F277W;,NIRCAM,SCIENCE,...,POLYGON ICRS 64.05090366707832 -24.09706124993...,-999,None,944.836,None,None,None,None,MACSJ0416.1-2403,INFRARED
2,-1,None,None,image,0.0,1005000000000,795000000000,F090W;F444W;,NIRCAM,SCIENCE,...,POLYGON ICRS 64.04628054402616 -24.09894062317...,-999,None,1374.307,None,None,None,None,MACSJ0416.1-2403,INFRARED
3,-1,None,None,image,0.0,1668000000000,1331000000000,F150W;F277W;,NIRCAM,SCIENCE,...,POLYGON ICRS 64.08333727261358 -24.12866890451...,-999,None,365.050,None,None,None,None,MACSJ0416.1-2403,INFRARED


We can also use our DataFrame to see a list of all the columns returned by the MAST query.

In [138]:
list(TEST_FRAME.columns)

['calib_level',
 'dataRights',
 'dataURL',
 'dataproduct_type',
 'distance',
 'em_max',
 'em_min',
 'filters',
 'instrument_name',
 'intentType',
 'jpegURL',
 'mtFlag',
 'objID',
 'objID1',
 'obs_collection',
 'obs_id',
 'obs_title',
 'obsid',
 'project',
 'proposal_id',
 'proposal_pi',
 'proposal_type',
 'provenance_name',
 's_dec',
 's_ra',
 's_region',
 'sequence_number',
 'srcDen',
 't_exptime',
 't_max',
 't_min',
 't_obs_release',
 'target_classification',
 'target_name',
 'wavelength_region']

We can use this list to further narrow our query and more closely match our own observations.

In [147]:
MY_FILTS = {"obs_collection": "JWST",
            "instrument_name": "NIRISS",
            "dataproduct_type": "spectrum",
            "filters": "F115W;GR150R",
           }
TEST_QUERY = filtered_position_query(MY_FILTS, SAMPLE_COORDS)
convert_to_dataframe(TEST_QUERY)

Returned 8 query results


,calib_level,dataRights,dataURL,dataproduct_type,distance,em_max,em_min,filters,instrument_name,intentType,...,s_region,sequence_number,srcDen,t_exptime,t_max,t_min,t_obs_release,target_classification,target_name,wavelength_region
0,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04177997317171 -24.09859778766...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
1,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.0418460661395 -24.098753197468...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
2,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04169386910588 -24.09879349014...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
3,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04163044640812 -24.09866367573...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
4,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04174391936442 -24.09852339959...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
5,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04181001225022 -24.09867880941...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
6,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04165781529366 -24.09871910205...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED
7,-1,None,None,spectrum,0,1300000000000,1000000000000,F115W;GR150R,NIRISS,SCIENCE,...,POLYGON ICRS 64.04159439266914 -24.09858928763...,-999,None,1202.518,None,None,None,None,MACSJ0416.1-2403,INFRARED


<a id="analyzing-results"></a>
<h3>Analyzing our results</h3>
<p>Now that we have some full query results available, we'll want to extract some of the relevant information.  First, we'll want to see a list of any nearby pointings found by our position query.  Next, we'll also want to see a list of which programs these pointings are associated with.</p>

In [12]:
def analyze_query_results(our_target, queryResults):
    """
    Perform analysis of the query results.  Identify the nearby planned pointings and
    identify the programs involved.
    
    :param our_target:  Expects a pair of coordinates in degrees.
    :type our_target:  tuple
    
    :param queryResults:  Full results from a MAST query.
    :type queryResults:  dictionary
    """
    
    # Set up initial variables
    data = queryResults['data']
    ra_target, dec_target = our_target
    targets = {}
    programs = []
    
    # Create a dictionary of all unique coordinate pairs along with a count of how many times they
    # are found
    for current in data:
        current_program = current['proposal_id']
        current_ra = current['s_ra']
        current_dec = current['s_dec']
        current_coords = (current_ra, current_dec)
        if current_coords in targets.keys():
            targets[current_coords] += 1
        else:
            targets[current_coords] = 1
            programs.append(current_program)

    # For each unique coordinate pair, calculate the distance from the target and display our 
    # results
    for x in sorted(targets.keys()):
        num_obs = targets[x]
        unique_ra, unique_dec = x
        result = "Found {0} planned observations at {1}, {2}".format(num_obs, 
                                                                     unique_ra, 
                                                                     unique_dec)
        distance_ra = abs(unique_ra - ra_target)
        distance_dec = abs(unique_dec - dec_target)
        distance = SkyCoord(distance_ra, distance_dec, frame="icrs", unit='deg')
        if distance_ra < 0.001 and distance_dec < 0.001:    # Account for rounding differences
            result += " (target match)"
        else:
            result += " ({0} away)".format(distance.to_string('hmsdms'))
        print(result)
        
    # Output a link for each program found
    for p in sorted(list(set(programs))):
        address = "https://jwst.stsci.edu/observing-programs/program-information?id={0}".format(p)
        print("Found planned observations in {0}: {1}".format(p, address))
        
    # Return the dictionary of coordinates and number of observations
    return targets

In [13]:
TEST_RESULTS = analyze_query_results(SAMPLE_COORDS, TEST_QUERY)

Found 13 planned observations at 64.0342, -24.0667138888889 (00h00m01.162s +00d00m20.33s away)
Found 36 planned observations at 64.03708333333333, -24.0746388888889 (00h00m00.47s +00d00m08.2s away)
Found 103 planned observations at 64.03904166666666, -24.07236111111109 (target match)
Found planned observations in 1176: https://jwst.stsci.edu/observing-programs/program-information?id=1176
Found planned observations in 1208: https://jwst.stsci.edu/observing-programs/program-information?id=1208


<p>Instead of simply listing associated proposal ID's, our module creates a link to the program information page for any associated proposals found in the area.  On these pages, the user may access additional information on the program or the associated APT file itself.  <b>Examining the information provided on these program pages is a critical step to investigating any potential sources of conflict for your JWST observations.</b></p>

<a id="save-to-file"></a>
<h3>Saving results to a file</h3>
<p>This gives us a basic idea of how many observations are currently planned in the vicinity, but this does not necessarily mean these observations will conflict.  We still need to check which instruments and configurations are being used.  Much of this information is contained in the query results we've obtained, and would be more easily digestable in a CSV table.</p>

In [14]:
import csv

def write_to_csv_file(queryResults, filename):
    """
    Write MAST query results to a .csv file.
    
    :param queryResults:  Full results from a MAST query.
    :type queryResults:  dictionary
    
    :param filename:  The desired filename to save the CSV table as.
    :type filename:  string
    """
    
    # Column names are stored in the 'fields' dictionary
    fields = queryResults['fields']
    header = [entry['name'] for entry in fields]

    # Use the DictWriter class to write the data dictionary to a .csv file
    directory = os.getcwd()
    filename = "/".join([directory, filename])
    data = queryResults['data']
    with open(filename, 'w') as output:
        writer = csv.DictWriter(output, fieldnames=header)
        writer.writeheader()
        w = [writer.writerow(obs) for obs in data]
        
    # Return the filename created
    print("Saved {0}".format(filename))
    return filename

In [15]:
# Choose a filename for the resulting CSV table
SAVE_AS = 'planned_obs_test.csv'
SAVED = write_to_csv_file(TEST_QUERY, SAVE_AS)

Saved /Users/pforshay/Documents/2003_jwst_duplication/planned_obs_test.csv


<a id="query-multi-targs"></a>
<h3>Process multiple targets</h3>
<p>We now have a CSV table with all parameters available through the Portal for all observations found within a 0.2 degree radius of a set of sample coordinates.  The API allows us to now take this one step further and bring all these modules together and check multiple sets of coordinates back-to-back.</p>

In [28]:
def check_multiple_targets(coordinates_list, coordinates_format, write=False):
    """
    Handle the full process for checking a list of targets.  This includes converting
    to degrees if necessary, sending an initial count query, sending the full MAST
    query, analyzing the results, and writing the results to a file.
    
    :param coordinates_list:  Expects a list of coordinate tuples, not necessarily in
                              degrees.
    :type coordinates_list:  list
    
    :param coordinates_format:  Which format the list of coordinates is provided in
                                will determine whether or not convert_to_degrees is run.
    :type coordinates_format:  string
    
    :param write:  Flag to save results to a .csv file.
    :type write:  boolean
    """
    
    results = []
    
    # Iterate through our list of coordinate tuples
    for target in coordinates_list:
        query_results = "None"
        print("...checking {0}...".format(target))
        
        # Convert each pair of coordinates to degrees if necessary
        if coordinates_format.lower() == "deg":
            in_degrees = target
        else:
            in_degrees = convert_to_degrees(target[0], target[1])
        
        # Submit an initial count query
        try:
            query_results = filtered_position_query(in_degrees)
        except RuntimeError as err:
            print(err)
            continue
            
        nearby_targets = analyze_query_results(in_degrees, query_results)
        
        # Generate a filename and write CSV table if 'write' enabled
        if write:
            filename = "results_{0}_{1}.csv".format(target[0], target[1])
            filename = write_to_csv_file(query_results, filename)
            
        results.append(query_results)
            
    # Return the list of all query result dictionaries
    return results

In [29]:
OUR_COORDINATES_LIST = [('04h16m09.370s', '-24d04m20.50s'),
                        ('05h42m15s', '+48d22m43s'),
                        ('19h45m01.190s', '-14d45m15.79s'),
                        ('20h20m20.20s', '+20d20m20.20s')
                       ]
OUR_QUERY_RESULTS = check_multiple_targets(OUR_COORDINATES_LIST, "icrs", write=False)    # Change the 'write' flag to save .csv files

...checking ('04h16m09.370s', '-24d04m20.50s')...
Found 13 planned observations at 64.0342, -24.0667138888889 (00h00m01.162s +00d00m20.33s away)
Found 36 planned observations at 64.03708333333333, -24.0746388888889 (00h00m00.47s +00d00m08.2s away)
Found 103 planned observations at 64.03904166666666, -24.07236111111109 (target match)
Found planned observations in 1176: https://jwst.stsci.edu/observing-programs/program-information?id=1176
Found planned observations in 1208: https://jwst.stsci.edu/observing-programs/program-information?id=1208
...checking ('05h42m15s', '+48d22m43s')...
No matching results found
...checking ('19h45m01.190s', '-14d45m15.79s')...
Found 48 planned observations at 296.2341625, -14.79758055555556 (00h00m04.991s +00d02m35.5s away)
Found 80 planned observations at 296.2420620833333, -14.8057277777778 (00h00m03.0951s +00d03m04.83s away)
Found 16 planned observations at 296.2437204166667, -14.7751777777778 (00h00m02.6971s +00d01m14.85s away)
Found planned observati

<a id="search-by-name"></a>
<h3>Getting coordinates from stationary target names</h3>
<p>Using the API also gives us access to the name-resolver service, which allows us to input a list of target names instead of coordinates.  The returned coordinates are already in degrees, so we can also skip the convert_to_degrees module.</p>

In [ ]:
def resolve_target_names(target_list):
    """
    Look up a list of target names using the MAST lookup service and return
    a list of coordinates.
    
    :param target_list:  The list of targets to be resolved.
    :type target_list:  list
    """
    
    # Set up an empty list for coordinate results and iterate through the list of target names
    coordinates_list = []
    for target_name in target_list:
        
        # Make a resolver request with the current target name
        resolverRequest = {'service':'Mast.Name.Lookup',
                           'params':{'input':target_name,
                                     'format':'json'}
                          }
        headers, resolvedObjectString = mastQuery(resolverRequest)
        resolvedObject = json.loads(resolvedObjectString)
        
        # If the target name was not found, we will run into IndexErrors when we try to set these 
        # variables
        try:
            target_ra = resolvedObject['resolvedCoordinate'][0]['ra']
            target_dec = resolvedObject['resolvedCoordinate'][0]['decl']
            canonical_name = resolvedObject['resolvedCoordinate'][0]['canonicalName']
        except IndexError:
            print("{0} not found".format(target_name))
            continue
            
        # Add the coordinates as a tuple to the list
        target_coords = (target_ra, target_dec)
        coordinates_list.append(target_coords)
        print("Found {0} at {1}".format(canonical_name, target_coords))
        
    # Return the list of coordinate tuples
    return coordinates_list

In [ ]:
OUR_TARGETS_LIST = ['M92',
                    '30 Doradus',
                    'Dumbbell Nebula',
                    'Fake Test Galaxy'
                   ]
NEW_COORDINATES_LIST = resolve_target_names(OUR_TARGETS_LIST)

<p>With our list of names resolved we can send our new list of coordinates to our previous query module.

In [ ]:
NEW_QUERY_RESULTS = check_multiple_targets(NEW_COORDINATES_LIST, "deg", write=False)    # Change the 'write' flag to save .csv files

<a id="moving-targets"></a>
<h3>Searching for moving targets (keyword searches on target_name)</h3>
<p>Moving targets are a bit trickier since we do not have a single set of coordinates to search against.  Instead, we'll look for matching entries in the target_name field and we'll use the freeText paramter with wildcards to find any close matches.  Again, we'll begin with an initial "COUNT_BIG(*)" query to get a results count first.</p>

In [ ]:
def filtered_keyword_query(keyword, count=True):
    """
    Construct a filtered mashup request to send to the mastQuery function.  This
    will search for a given keyword within the 'target_name' field and return
    any results including wildcards.
    
    :param keyword:  The keyword to search for within 'target_name'.
    :type keyword:  string
    
    :param count:  Flag to submit a full query or a count query.  Sends count
                   by default.
    :type count:  boolean
    """
    
    # Use count flag to determine the columns queried
    if count:
        columns = "COUNT_BIG(*)"    # "COUNT_BIG(*)" will only get a count of the results
    else:
        columns = "*"
        
    # Construct the mashup request
    service = "Mast.Caom.Filtered"
    filters = [{"paramName":"calib_level", "values":["-1"]},
               {"paramName":"obs_collection", "values":["JWST"]},
               {"paramName":"target_name", "values":[],"freeText":"%"+keyword+"%"}
              ]
    mashupRequest = {"service":service,
                     "format":"json",
                     "params":{"columns":columns,
                               "filters":filters
                              }
                    }

    # Submit the MAST query
    headers,outString = mastQuery(mashupRequest)
    queryResults = json.loads(outString)
    
    # Return the results, depending on the type of query submitted
    if count:
        data = queryResults['data']
        count = data[0]['Column1']
        return count
    else:
        return queryResults

In [ ]:
JUPITER_COUNT = filtered_keyword_query("Jupiter")
print(JUPITER_COUNT)

<p>We find 126 target names containing "Jupiter", so we'll go ahead and submit the full query.</p>

In [ ]:
JUPITER_QUERY = filtered_keyword_query("Jupiter", count=False)
display_results(JUPITER_QUERY, 4)

<p>With query results in hand we'll build a module to analyze our moving target output.  We'll want lists of the target names we matched with and which programs these are found in.</p>

In [ ]:
def keyword_matches(queryResults):
    """
    Create a list of the unique target names that were returned by the filtered_keyword_query.
    Also generate a link to each nearby program found.
    
    :param queryResults:  Full results from a MAST query.
    :type queryResults:  dictionary
    """
    
    # Set up initial variables
    data = queryResults['data']
    targets = []
    programs = []
    
    # For each query result, pull out the target_name and proposal_id values
    for obs in data:
        current_target = obs['target_name']
        current_program = obs['proposal_id']
        targets.append(current_target)
        programs.append(current_program)
        
    # Get the unique sets of target_name and proposal_id values
    unique_targets = sorted(list(set(targets)))
    unique_programs = sorted(list(set(programs)))
    
    # Output our results
    print("Matched the keyword to: {0}".format(unique_targets))
    for p in unique_programs:
        address = "https://jwst.stsci.edu/observing-programs/program-information?id={0}".format(p)
        print("Found planned observations in {0}: {1}".format(p, address))
        
    # Return the list of unique target names
    return unique_targets

In [ ]:
JUPITER_MATCHES = keyword_matches(JUPITER_QUERY)

<p>Copying our multiple targets module from above, with a few tweaks we can now run back-to-back queries on a list of moving target names.  We can also incorporate the write_to_csv_file module to automatically save all our results for further inspection.</p>

In [ ]:
def check_multiple_keywords(keywords_list, write):
    """
    Perform multiple filtered MAST queries based on a list of keywords to look
    for within the 'target_name' field.  Analyze the results and write them to
    a .csv file if requested.
    
    :param keywords_list:  List of target keywords to search for in the 'target_name'
                           field.
    :type keywords_list:  list
    
    :param write:  Flag whether or not the results are save in a .csv file.
    :type write:  boolean
    """
    
    results = []
    
    # Iterate through our list of target_name keywords
    for target in keywords_list:
        query_results = "None"
        print("...checking {0}...".format(target))
        
        # Submit an initial count query
        count = filtered_keyword_query(target)
        
        # If the count is within a valid range, submit the full query
        if count > 0 and count < 50000:
            query_results = filtered_keyword_query(target, count=False)
            targ_names = keyword_matches(query_results)
            
            # Generate a filename and write CSV table if 'write' enabled
            if write:
                filename = "results_{0}.csv".format(target)
                filename = write_to_csv_file(query_results, filename)
            
        # Skip if too many results are found
        elif count > 50000:
            print("More than 50,000 results found!  Please narrow your query.")
            
        # Skip if no results are found
        elif count == 0:
            print("No target_name matches found for {0}".format(target))
            
        results.append(query_results)
        
    # Return the list of query result dictionaries
    return results

In [ ]:
OUR_MOVING_TARGETS = ["Jupiter",
                      "Titan",
                      "Ceres",
                      "Sun"
                     ]
MOVING_MATCHES = check_multiple_keywords(OUR_MOVING_TARGETS, write=False)    # Change the 'write' flag to save .csv files

<a id="send-to-aladin"></a>
<h3>Visualizing results with Aladin</h3>
<p>Now that we have some idea which of our targets may have planned JWST observations nearby, it may help to plot the existing planned observations in the area.  The <a href="https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html" target="_blank">MAST Portal</a> handles this automatically, but we can also make a simple display with footprints here using Aladin Lite.  HOWEVER, these footprints do not currently take into account some factors such as telescope rotation/orientation, or offsets from the main pointing.  These footprints only represent an approximation of the instrument FOV's.</p>
<p>First, we'll need a function to extract the provided region information from our previous query results.</p>

In [ ]:
def get_polygons(queryResults):
    """
    Extract 's_region' information from a set of MAST query results.  Only
    returns the unique set of footprints found.
    
    :param queryResults:  Full results from a MAST query.
    :type queryResults:  dictionary
    """
    
    # We're only interested in the 'data' dictionary from the query results here
    data = queryResults['data']
    polygons = []
    
    # Iterate through each observation
    for obs in data:
        region = obs['s_region']
        individual = region[7:].split(" ")    # Cut off prepending 'POLYGON'
        individual = list(filter(None, individual))    # Eliminate empty entries from extra spaces
        shape = []
        
        # Pair off the entries into RA, Dec coordinates
        while len(individual) > 1:
            coordinate = [individual[0], individual[1]]
            shape.append(coordinate)
            individual = individual[2:]
            
        # We can ignore repeated footprints
        if shape in polygons:
            continue
        else:
            print(shape)
            polygons.append(shape)
            
    # Return the list of coordinate lists
    return polygons

In [ ]:
RESULTS_TO_PLOT = OUR_QUERY_RESULTS[2]
TEST_REGIONS = get_polygons(RESULTS_TO_PLOT)

<p>With our coordinates and footprint information, we'll construct a string to embed an Aladin Lite script into HTML.</p>

In [ ]:
def make_aladin_html(coordinates, footprints=None):
    """
    Construct an HTML-formatted string to embed an Aladin Lite viewer script,
    which will center on a set of given coordinates and draw any provided
    footprints in overlays.
    
    :param coordinates:  A single set of coordinates in degrees.
    :type coordinates:  tuple
    
    :param footprints:  A list of any polygon coordinates found in the 's_region'
                        field from MAST query results.
    :type footprints:  list
    """
    
    # Format an RA/Dec string for Aladin
    ra = str(coordinates[0])
    dec = str(coordinates[1])
    coords = ra + ", " + dec
    
    # Create a unique div ID, allowing for multiple windows
    div = "aladin-lite-" + ra.replace(".", "") + dec.replace(".", "")
    
    # Begin constructing the HTML string with Aladin embedded
    aladin_string = """<!-- include Aladin Lite CSS file in the head section of your page -->
    <link rel="stylesheet" href="//aladin.u-strasbg.fr/AladinLite/api/v2/latest/aladin.min.css" />

    <!-- you can skip the following line if your page already integrates the jQuery library -->
    <script type="text/javascript" src="//code.jquery.com/jquery-1.12.1.min.js" charset="utf-8"></script>

    <!-- insert this snippet where you want Aladin Lite viewer to appear and after the loading of jQuery -->
    <div id='""" + div + """' style="width:800px;height:800px;"></div>
    <script type="text/javascript" src="//aladin.u-strasbg.fr/AladinLite/api/v2/latest/aladin.min.js" charset="utf-8"></script>
    <script type="text/javascript">
        var aladin = A.aladin('#""" + div + """', {survey: "P/DSS2/color", fov:0.5, target: '""" + coords + """'});
        var overlay = A.graphicOverlay({color: '#ee2345', lineWidth: 3});
        aladin.addOverlay(overlay);
    """

    # Add an overlay string for each footprint found
    for region in footprints:
        footprint = str(region)
        overlay = "overlay.addFootprints(A.polygon({0}));".format(footprint)
        aladin_string += overlay
        
    aladin_string += "</script>"
    
    # Return the full constructed string
    return aladin_string

In [ ]:
COORDINATES_TO_PLOT = OUR_COORDINATES_LIST[2]
PLOT_IN_DEGREES = convert_to_degrees(COORDINATES_TO_PLOT[0], COORDINATES_TO_PLOT[1])
ALADIN_HTML = make_aladin_html(PLOT_IN_DEGREES, footprints=TEST_REGIONS)

<p>Next, we'll need to access IPython's core library for running HTML code inside a notebook.  Then we can use that to execute the HTML code we constructed and view the resulting plot, which retains Aladin Lite's pan and zoom capabilities as well as the 'Export view as PNG' feature.</p>

In [ ]:
from IPython.core.display import HTML
HTML(ALADIN_HTML)